In [1]:
import ssl
import sys
import psycopg2
from sqlalchemy import create_engine
import paho.mqtt.client
import json
import pandas as pd

In [2]:
def pretty_select(query):
    connection_string = 'postgres://{}:{}@{}:5432/{}'.format(user, password, host, dbname)
    try:
        engine = create_engine(connection_string)
        records = pd.read_sql_query(query, engine)
    except Exception as e:
        print('Error en el query:', e)
    else:
        return records

In [3]:
def select(query):
    cur = myConnection.cursor()
    try:
        cur.execute(query)
    except Exception as e:
        myConnection.commit()
        print('Error en el query:', e)
    else:
        records = cur.fetchall()
        cur.close()
        return records

In [4]:
host='ruby.db.elephantsql.com'
user ='uicdhpnp'
password='Kfp61NZwnYQVCSDf-zl7Jae836R2u0Fn'
dbname='uicdhpnp'

In [21]:
query = """insert into Afiliado (ID_Cliente, puntos, fecha) values (25021709, 73115422, '8/7/2019');
insert into Afiliado (ID_Cliente, puntos, fecha) values (25399720, 48326798, '7/1/2018');
insert into Afiliado (ID_Cliente, puntos, fecha) values (26044929, 29777845, '7/4/2018');
insert into Afiliado (ID_Cliente, puntos, fecha) values (26118740, 64145287, '2/21/2019');
insert into Afiliado (ID_Cliente, puntos, fecha) values (24997875, 16701050, '7/12/2018');
insert into Afiliado (ID_Cliente, puntos, fecha) values (25619359, 91948793, '1/19/2019');
insert into Afiliado (ID_Cliente, puntos, fecha) values (24776966, 61027902, '2/19/2020');
insert into Afiliado (ID_Cliente, puntos, fecha) values (27205181, 77602299, '5/22/2020');
insert into Afiliado (ID_Cliente, puntos, fecha) values (27651832, 97954860, '9/7/2018');
insert into Afiliado (ID_Cliente, puntos, fecha) values (25300265, 79125628, '12/31/2019');
insert into Afiliado (ID_Cliente, puntos, fecha) values (27408935, 3980860, '9/2/2018');
insert into Afiliado (ID_Cliente, puntos, fecha) values (24846813, 68522885, '11/13/2019');
insert into Afiliado (ID_Cliente, puntos, fecha) values (24714462, 31311721, '10/20/2018');
insert into Afiliado (ID_Cliente, puntos, fecha) values (27669949, 38812618, '5/15/2019');
insert into Afiliado (ID_Cliente, puntos, fecha) values (25259010, 2204526, '11/5/2018');"""

cur = myConnection.cursor()
cur.execute(query)

In [ ]:
myConnection.close()

In [36]:
myConnection = psycopg2.connect(host = host,
                                user= user, password =password,
                                dbname= dbname)

In [37]:
query = """create table sensores_data (
	ID_Estante varchar (4),
    cant numeric (3,0),
    temp numeric (3,1),
    hora character varying,
    primary key(ID_Estante, hora)
);"""
cur = myConnection.cursor()
cur.execute(query)
cur.close()
myConnection.commit()

In [32]:
query = """insert into Factura (ID_Factura, fecha, ID_Banco, ID_Compra) values (4001, '10/24/2019', 2, 6001);
insert into Factura (ID_Factura, fecha, ID_Banco, ID_Compra) values (4002, '8/30/2019', 1, 6001);"""

cur = myConnection.cursor()
cur.execute(query)

In [270]:
query = """UPDATE fac
 SET fac.total = c.suma
 FROM factura fac
 INNER JOIN
 (
   SELECT ID_Compra, SUM(cant) suma
   FROM compra
   GROUP BY ID_Compra
 ) c ON fac.ID_Compra = c.ID_Compra"""

In [33]:
query = """select * from factura"""
select(query)

[('4001', None, datetime.date(2019, 10, 24), '2', '6001'),
 ('4002', None, datetime.date(2019, 8, 30), '1', '6001')]

In [ ]:
query = """update sucursal set max_capacidad = 100 where ID_Sucursal like '1001'"""
cur = myConnection.cursor()
cur.execute(query)
cur.close()
myConnection.commit()

In [91]:
query = """select * from cliente  full join afiliado on cliente.ID_Cliente = afiliado.ID_Cliente 
            where cliente.ID_Cliente is null or afiliado.ID_Cliente is null"""
select(query)

[('24403879',
  'Orelee',
  'Hilbourne',
  datetime.date(1988, 12, 20),
  None,
  None,
  None),
 ('27589249',
  'Candice',
  'Dinneges',
  datetime.date(1978, 3, 10),
  None,
  None,
  None),
 ('25908283', 'Ev', 'Thurborn', datetime.date(1977, 12, 16), None, None, None),
 ('28893227',
  'Neilla',
  'Andell',
  datetime.date(1983, 2, 23),
  None,
  None,
  None),
 ('27459306',
  'Patience',
  'Spinelli',
  datetime.date(1977, 3, 13),
  None,
  None,
  None),
 ('28157623',
  'Amitie',
  'Willowby',
  datetime.date(1994, 4, 20),
  None,
  None,
  None),
 ('28861799',
  'Annamarie',
  'Laurance',
  datetime.date(1982, 4, 26),
  None,
  None,
  None),
 ('25531704',
  'Hurlee',
  'Lambourn',
  datetime.date(1985, 4, 14),
  None,
  None,
  None),
 ('24490605', 'Alair', 'Yeo', datetime.date(1991, 12, 31), None, None, None)]

In [ ]:
#Cuales son los clientes conocidos y desconocidos que mas han comprado en cada una de las tiendas

In [ ]:
#Diga cual es la categoría de producto que menos se ha vendido en cada una de las tiendas.

In [ ]:
#Cuales es el top 5 de productos que mas se han vendido en cada una de las tiendas.

In [ ]:
#Diga todos los clientes que en los últimos 7 días a partir de la ejecución del query han comprado
#solo en una tienda y los que han comprado en las dos tiendas (realice queries distintos).

In [ ]:
#Haga un análisis por estante de cuales categorías de producto tienen mayor rotación de
#inventario y haga una propuesta de modificación

In [ ]:
#Encuentre a todos los clientes que han pagado con 2 bancos distintos en la ultima semana. 
#Enumérelos y diga si son parte del programa de afiliados.

In [ ]:
#Realizar compra: Esto procedimiento almacenado debe tener como parámetro una lista de
#productos con su cantidad, el banco por donde se va a cancelar y un cliente. Debe tener toda la
#lógica para ingresar en las distintas tablas la compra realizada.

In [ ]:
#Haga un procedimiento almacenado que al final del día saque un estado de cuenta de cuanto
#dinero se tiene en cada banco

In [ ]:
#Haga un procedimiento almacenado que al final del mes saque un estado de cuenta de los
#puntos de todos los afiliados al programa de fidelidad que compraron ese mes.

In [ ]:
#Cada vez que entre un cliente conocido, se debe agregar 1 punto a su estado de cuenta del
#programa de fidelidad.

In [ ]:
#Cada vez que un cliente conocido realice una compra, se le agregaran los puntos en su programa
#de fidelidad

In [ ]:
#Si un cliente desconocido con la misma cedula compra mas de 4 veces, automáticamente se
#agregará al programa de fidelidad.